In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [6]:
transform = transforms.Compose([transforms.ToTensor()])

mnist_train = datasets.MNIST(root='data/',download=True, train=True, transform=transform)
mnist_test = datasets.MNIST(root='data/', download=True, train=False, transform=transform)


0it [00:00, ?it/s]

100%|█████████▉| 9871360/9912422 [02:32<00:00, 63354.38it/s] 

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



0it [00:00, ?it/s]


 57%|█████▋    | 16384/28881 [00:05<00:01, 9954.23it/s]
32768it [00:05, 5567.80it/s]                           
0it [00:00, ?it/s]

...


 25%|██▌       | 417792/1648877 [00:10<00:29, 41697.29it/s]
9920512it [02:50, 63354.38it/s]                             [A
100%|█████████▉| 1646592/1648877 [00:24<00:00, 80261.48it/s]

0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




  0%|          | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 10861.51it/s]            

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!



1654784it [00:41, 80261.48it/s]                             

In [140]:
#Loading dataset into dataloader
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False)

In [141]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
     
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.5)

        self.fc1 = nn.Linear(128*7*7, 50,bias=True)
        
        self.fc2 = nn.Linear(50, 10,bias=True)


    def forward(self, x):
        
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)

        out = self.dropout(out)

        
        out = self.fc1(out)
        out = self.sigmoid(out)
        out = self.fc2(out)
        out = self.softmax(out)

        
        return out


In [142]:
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()


model = CNN()


optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()


In [143]:
training_epochs = 15
total_batch = len(mnist_train) // batch_size

def compute_accuracy(Y_target, hypothesis):
    _, predicted = torch.max(hypothesis.data, 1)
    total = Y_target.size(0)
    correct = (predicted == Y_target).sum().item()

    return correct/total


print("Training Started")

for epoch in range(training_epochs):
    avg_cost = 0
    for i, (X, Y) in enumerate(train_loader):
        optimizer.zero_grad()

        hypothesis = model(X)

        cost = criterion(hypothesis, Y)

        cost.backward()

        optimizer.step()

        accu = compute_accuracy(Y, hypothesis)
 
        if i % 200 == 0:
            print("Epoch= {},\t batch = {},\t cost = {:.4f},\t accuracy = {}".format(epoch, i, cost.item(), accu))

        avg_cost += cost.data / total_batch

    print("Epoch: {}, averaged cost = {:.4f}".format(epoch + 1, avg_cost.item()))


print('Learning Finished!')

Training Started
Epoch= 0,	 batch = 0,	 cost = 2.3001,	 accuracy = 0.03125
Epoch= 0,	 batch = 200,	 cost = 1.7316,	 accuracy = 0.90625
Epoch= 0,	 batch = 400,	 cost = 1.5455,	 accuracy = 0.96875
Epoch= 0,	 batch = 600,	 cost = 1.5328,	 accuracy = 0.9375
Epoch= 0,	 batch = 800,	 cost = 1.5054,	 accuracy = 1.0
Epoch= 0,	 batch = 1000,	 cost = 1.4924,	 accuracy = 1.0
Epoch= 0,	 batch = 1200,	 cost = 1.5321,	 accuracy = 0.9375
Epoch= 0,	 batch = 1400,	 cost = 1.4705,	 accuracy = 1.0
Epoch= 0,	 batch = 1600,	 cost = 1.4761,	 accuracy = 1.0
Epoch= 0,	 batch = 1800,	 cost = 1.4755,	 accuracy = 1.0
Epoch: 1, averaged cost = 1.5634
Epoch= 1,	 batch = 0,	 cost = 1.5254,	 accuracy = 0.9375
Epoch= 1,	 batch = 200,	 cost = 1.5057,	 accuracy = 0.96875
Epoch= 1,	 batch = 400,	 cost = 1.4632,	 accuracy = 1.0
Epoch= 1,	 batch = 600,	 cost = 1.4772,	 accuracy = 1.0
Epoch= 1,	 batch = 800,	 cost = 1.4925,	 accuracy = 0.96875
Epoch= 1,	 batch = 1000,	 cost = 1.5023,	 accuracy = 0.96875
Epoch= 1,	 batch = 

Epoch= 13,	 batch = 1000,	 cost = 1.4613,	 accuracy = 1.0
Epoch= 13,	 batch = 1200,	 cost = 1.4862,	 accuracy = 0.96875
Epoch= 13,	 batch = 1400,	 cost = 1.4612,	 accuracy = 1.0
Epoch= 13,	 batch = 1600,	 cost = 1.4614,	 accuracy = 1.0
Epoch= 13,	 batch = 1800,	 cost = 1.4742,	 accuracy = 1.0
Epoch: 14, averaged cost = 1.4684
Epoch= 14,	 batch = 0,	 cost = 1.4612,	 accuracy = 1.0
Epoch= 14,	 batch = 200,	 cost = 1.4635,	 accuracy = 1.0
Epoch= 14,	 batch = 400,	 cost = 1.4612,	 accuracy = 1.0
Epoch= 14,	 batch = 600,	 cost = 1.4612,	 accuracy = 1.0
Epoch= 14,	 batch = 800,	 cost = 1.4612,	 accuracy = 1.0
Epoch= 14,	 batch = 1000,	 cost = 1.4889,	 accuracy = 0.96875
Epoch= 14,	 batch = 1200,	 cost = 1.4626,	 accuracy = 1.0
Epoch= 14,	 batch = 1400,	 cost = 1.4614,	 accuracy = 1.0
Epoch= 14,	 batch = 1600,	 cost = 1.4613,	 accuracy = 1.0
Epoch= 14,	 batch = 1800,	 cost = 1.4612,	 accuracy = 1.0
Epoch: 15, averaged cost = 1.4683
Learning Finished!


In [144]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images)
        labels = Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 99.15 %
